In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# OscFit path
sys.path.append('/home/jpyanez/projects/rpp-kenclark/jpyanez/oscfit_code/oscfit_kpi/modules')

In [3]:
import dataLoader, oscFit
reload(dataLoader)
import MCEqFlux

In [4]:
user = 'kpi_user_full'
pid_array = np.array([-3, 2., np.inf])
sky_start = -1
sky_stop = 1
bins = 9

In [5]:
loader =  dataLoader.dataLoader(bin_edges   = [10**np.linspace(0.75,2.15,11),
                                               np.arccos(np.linspace(sky_start,sky_stop,bins))[::-1],
                                               pid_array],
                                observables = ['reco_energy', 'reco_zenith', 'delta_llh'],
                                user = user,
                                LEaxis = [],
                                #bkg_diff_is_syst = True, # Needs implementation
                                weight_keys = ['weight_e', 'weight_mu'],
                                detailed_detsys=False, # Re-calculate the detector variations at each minimization step
                                verbose = True)


 ************************************ 
 **** oscFit3D v1.1 - dataLoader ****
 ************************************

dataLoader: Initializing user kpi_user_full
dataLoader: Trying to load systematic histograms from file: 
/home/jpyanez/icecube/icecube-project/oscfit_results/systematic_functions/kpi_user_full_detSysHist.pckl
Reading from file
dataLoader: MC loaded successfully!
dataLoader: Start loading muon MC (baseline_only= True , use_kde= False , quantile= (0.0, 1.0) ).
dataLoader: Muon background templates loaded successfully!
dataLoader: Muon background templates loaded successfully!
Different simulation was selected, forcing reweighting
New simulation :  baseline
Different simulation was selected, forcing reweighting
New simulation :  baseline
Different simulation was selected, forcing reweighting
New simulation :  baseline
dataLoader: Combining systematics for  nue
dataLoader: Combining systematics for  numu
dataLoader: Combining systematics for  nutau
dataLoader: Combining syst

In [14]:
print 'DONE'

DONE


In [15]:
livetime = loader.sec2years*2.8

In [16]:
import oscFit
reload(oscFit)

<module 'oscFit' from '/home/jpyanez/projects/rpp-kenclark/jpyanez/oscfit_code/oscfit_kpi/modules/oscFit.pyc'>

In [17]:
norm_atmmu = 1.4
ref = loader.loadMCasData(settings = {'norm_nu':2.9,
                          'norm_atmmu':norm_atmmu,
                          'atmmu_template':'data',
                          'oscMode':'TwoNeutrino',
                          'add_detector_systematics':False})
tot_events = np.sum(ref)
print 'Events in pseudo-data sample', np.sum(ref)

dataLoader: Loading MC as data. Calculating weights with the parameters that follow:
{'theta13': 0.155, 'noise_f': None, 'epsilon_ee': 0.0, 'sigma_gamma_mu': 0.0, 'hi_fwd': 0.0, 'add_detector_systematics': False, 'epsilon_mutau': 0.0, 'oscMode': 'TwoNeutrino', 'theta24': 0.0, 'atmmu_f': None, 'atmmu_template': 'data', 'theta23': 0.74, 'dm31': 0.0025, 'uphor_ratio': 0.0, 'nubar_diff_dis': 0.0, 'spe_correction': 0.0, 'spiciness': 0.0, 'axm_res': 0.0, 'norm_tau': 1.0, 'pid_bias': 0.0, 'norm_nu': 2.9, 'fluxTables': False, 'oscTables': False, 'nubar_ratio': 0.0, 'nu_diff_dis': 0.0, 'nu_nubar': 1.0, 'norm_nc': 1.0, 'domeff': 1.0, 'norm_atmmu': 1.4, 'hole_ice': 0.25, 'norm_noise': 1.0, 'dis_reweight': False, 'dm41': 0.0, 'correct_honda_flux': False, 'had_escale': 1.0, 'theta34': 0.0, 'mix_angle': 1.0, 'simulation': 'baseline', 'norm_e': 1.0, 'axm_qe': 0.0, 'epsilon_etau': 0.0, 'epsilon_mumu': 0.0, 'epsilon_emu': 0.0, 'ma_variations': True, 'epsilon_tautau': 0.0, 'apply_baseline_correction': T

In [18]:
ini_fraction = loader.atmmu_histo['data'].sum()*norm_atmmu/tot_events
print ini_fraction

0.06556761643464981


In [19]:
fitter = oscFit.fitOscParams()
data_loaders    = [loader]          # Here you could list many samples to be fit at once
data_histograms = [ref]  # The data histogram for each of the samples


 ************************************ 
 ****** oscFit3D v1.1 - oscFit ******
 ************************************



In [20]:
# The settings of the fit are given as [ini_value, fix?]
fit_settings    = {
    'simulation':     'baseline',      # This patches GENIE simulation with NuGen.
    'dm31':           [0.0025, False, 'NH'], # Need to specify the hierarchy of the fit
    'theta23':        [0.76, False],         # Not used in the "TwoNeutrino" mode
    'theta13':        [0.148, True],        # Not used in the "TwoNeutrino" mode
    'mix_angle':      [0.97, False, 1.4],     # Only used in the "TwoNeutrino" mode. Leave as is otherwise.
    'oscMode':        'TwoNeutrino',         # How do you want to fit the data?
    'oscTables':      False,
    'norm':           [1.0, False],
    'norm_e':         [1., True],
    'norm_tau':       [1., True],            # Set to false for nutau apperance studies
    'nu_nubar':       [1., True],
    'nubar_ratio':     [0., True],
    'uphor_ratio':    [0., True],

    'gamma':          [0.00, False],
    # New parameters
    #'w_pi':           [0.00, True],
    #'w_k':            [0.02, False],
    
    
    'axm_qe':         [0., True],
    'axm_res':        [0., False],
    'pid_bias':       [0., True], #Highly experimental
    'hole_ice':       [0.02, False],
    'hi_fwd':         [0.0, False],

    'norm_nc':        [1., False],
    'domeff':         [1., False],
    'had_escale':     [1., True], # Highly experimental
    'atmmu_f':        [0.07, False, 'data'],
    'noise_f':        [0.0, True],
#    'fix_norm_region':[1.7],
    'detector_syst':  False,
    'include_priors': True,
    'fit_function': 'chi_squared',
    'printMode':      -1}                  # Do you want to see every step MINUIT takes?

In [21]:
result_NH = fitter(data_histograms=data_histograms,
                data_loaders=data_loaders,
                fit_settings=fit_settings,
                #fit_priors = fit_priors,
                ncalls = 1000,
                #do_chi2_fit = True,
                store_fit_details = True)

Prior: axm_res [0.0, 1.0]
Prior: had_escale [1.0, 0.1]
Prior: sigma_gamma_mu [0.0, 1.0]
Prior: nubar_diff_dis [0, 0.1008]
Prior: nubar_ratio [0.0, 1.0]
Prior: axm_qe [0.0, 1.0]
Prior: nu_diff_dis [0, 0.0757]
Prior: norm_e [1.0, 0.05]
Prior: uphor_ratio [0.0, 1.0]
Prior: norm_nc [1.0, 0.2]
Prior: gamma [0.05, 0.1]
Prior: domeff [1.0, 0.1]
Prior: theta13 [0.155, 0.008]
Prior: hole_ice [0.25, 0.1]
oscFit: Fitter running in TwoNeutrino mode
oscFit: 52568.63353322245  events in data
oscFit: Fitting in normal hierarchy mode
oscFit: found octant in settings B
found octant constrain to theta23 as (0.0, 1.5707963267948966)
7.94006
oscFit: Testing the LLH function  7.940063667227744
oscFit: Fit settings
	noise_f 	[0.0, True]
	epsilon_ee 	[0.0, True]
	sigma_gamma_mu 	[0.0, True]
	octant 	B
	hi_fwd 	[0.0, False]
	blind_keys 	[]
	printMode 	-1
	epsilon_mutau 	[0.0, True]
	oscMode 	TwoNeutrino
	include_priors 	True
	theta24 	[0.0, True]
	atmmu_f 	[0.07, False, 'data']
	dm31 	[0.0025, False, 'NH']
	t

6.29961  +0.00250  +0.99980  +0.99995  +0.06551  +0.00083  +0.99943  +0.02335
6.29958  +0.00250  +0.99980  +0.99995  +0.06551  +0.00083  +0.99943  +0.02335
6.29958  +0.00250  +0.99983  +0.99995  +0.06551  +0.00083  +0.99943  +0.02335
6.29962  +0.00250  +0.99977  +0.99995  +0.06551  +0.00083  +0.99943  +0.02335
6.29958  +0.00250  +0.99980  +0.99996  +0.06551  +0.00083  +0.99943  +0.02335
6.29962  +0.00250  +0.99980  +0.99994  +0.06551  +0.00083  +0.99943  +0.02335
6.29957  +0.00250  +0.99980  +0.99995  +0.06551  +0.00084  +0.99943  +0.02335
6.29963  +0.00250  +0.99980  +0.99995  +0.06551  +0.00082  +0.99943  +0.02335
6.29957  +0.00250  +0.99980  +0.99995  +0.06552  +0.00083  +0.99943  +0.02335
6.29963  +0.00250  +0.99980  +0.99995  +0.06551  +0.00083  +0.99943  +0.02335
6.29956  +0.00250  +0.99980  +0.99995  +0.06551  +0.00083  +0.99943  +0.02375
6.29964  +0.00250  +0.99980  +0.99995  +0.06551  +0.00083  +0.99943  +0.02294
6.29961  +0.00250  +0.99980  +0.99995  +0.06551  +0.00083  +0.99

6.29522  +0.00250  +0.99998  +1.00000  +0.06560  +0.00210  +0.99963  +0.07597
6.29522  +0.00250  +0.99998  +1.00000  +0.06559  +0.00210  +0.99963  +0.07638
6.29522  +0.00250  +0.99998  +1.00000  +0.06559  +0.00210  +0.99975  +0.07597
6.29522  +0.00250  +0.99998  +1.00000  +0.06560  +0.00209  +0.99963  +0.07638
6.29522  +0.00250  +0.99998  +1.00000  +0.06560  +0.00209  +0.99975  +0.07597
6.29522  +0.00250  +0.99998  +1.00000  +0.06559  +0.00209  +0.99975  +0.07638
6.29522  +0.00250  +0.99998  +1.00000  +0.06559  +0.00209  +0.99963  +0.07597
6.29534  +0.00250  +0.99998  +1.00000  +0.06559  +0.00209  +0.99963  +0.07597
6.29534  +0.00250  +0.99998  +1.00000  +0.06559  +0.00209  +0.99963  +0.07597
6.29522  +0.00250  +0.99998  +1.00000  +0.06559  +0.00209  +0.99963  +0.07597
6.29522  +0.00250  +0.99998  +1.00000  +0.06559  +0.00209  +0.99963  +0.07597
6.29534  +0.00250  +1.00024  +1.00000  +0.06559  +0.00209  +0.99963  +0.07597
6.29534  +0.00250  +0.99972  +1.00000  +0.06559  +0.00209  +0.99

TypeError: self._state cannot be converted to a Python object for pickling

In [ ]:
print 'DONE'

In [ ]:
params = []
for param in result_NH["parameters"]:
    if not result_NH["fit_settings"][param][1]:
        params.append(param)
ticks = range(0,len(params))
fig, ax = plt.subplots()
im = ax.imshow(np.transpose(result_NH["corr_matrix"]),cmap='RdBu_r', aspect='auto', interpolation='none', vmin=-1.0, vmax=1.0)
fig.colorbar(im, ax=ax)
ax.grid(False)
ax.set_yticks(ticks)
ax.set_xticks(ticks)
ax.set_xticklabels(params)
ax.set_yticklabels(params)
plt.xticks(rotation=90)
#x0, x1, y0, y1 = plt.axis()
#plt.axis((x0+3.0,x1+3.0, y0, y1))
fig.tight_layout()